In [1]:
import pymorphy2
import requests
from bs4 import BeautifulSoup
import re
from stop_words import get_stop_words

import numpy as np
import pandas as pd
from operator import itemgetter

### Скачивание страниц и нормализация предложений

In [2]:
class NormalAnalyzer:
    __stemmer = pymorphy2.MorphAnalyzer()
    __cache = {}
    
    def norm(self, token):
        token = token.strip()
        
        norm = ''
        if token not in self.__cache:
    
            res = self.__stemmer.parse(token)
            norm = res[0].normal_form
            self.__cache[token] = norm
        else:
            norm = self.__cache[token]
        return norm

In [3]:
class TextUrl():
    def __init__(self):
        self.sentensesDetect = re.compile(r"[^.!?\s][^.!?]*(?:[.!?](?!['\"]?\s|$)[^.!?]*)*[.!?]?['\"]?(?=\s|$)", re.MULTILINE |  re.DOTALL )
        self.wordsDetect = re.compile(u"[А-Яа-я]+")
        
        self.normal_analizer = NormalAnalyzer()
        self.stop_words = get_stop_words('ru')
        
    def sentenseToWords(self, sent):
        words = self.wordsDetect.findall(sent)
        normal_words = []

        # lemmatize words
        for word in words:
            word = self.normal_analizer.norm(word.lower())
            if word not in self.stop_words:
                normal_words.append(word)
        
        return normal_words
        
    def extractText(self, url):
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        
        sentenses = []
        realSentenses = []
        for paragraph in soup.find_all('p'):
            text = paragraph.get_text()
            for sent in self.sentensesDetect.findall(text):
                normal_words = self.sentenseToWords(sent)
                
                if len(normal_words) != 0:
                    sentenses.append(normal_words)
                    realSentenses.append(sent)
        
        return sentenses, realSentenses

In [4]:
textUrl = TextUrl()
all_sentenses = []
all_real_sentenses = []
all_urls = ['https://ru.wikipedia.org/wiki/Рабье,_Бенжамен',
            'https://ru.wikipedia.org/wiki/Весёлая_бурёнка',
            'https://ru.wikipedia.org/wiki/Норвежский_чёрный_элкхунд',
            'https://ru.wikipedia.org/wiki/Космические_Юра_и_Нюра',
            'https://ru.wikipedia.org/wiki/Невесомость',
            'https://ru.wikipedia.org/wiki/Союз_МС-04'
           ]
for url in all_urls:
    sent, realSents = textUrl.extractText(url)
    all_sentenses.extend(sent)
    all_real_sentenses.extend(realSents)

### Функции над векторами, представленными словарями

In [5]:
def lenVector(vec):
    return np.linalg.norm(vec.values())

In [6]:
def normolizeVec(vec):
    lenVec = lenVector(vec)
    if lenVec == 0:
        return 0
    return dict(zip(vec.keys(), vec.values() / lenVec))

In [7]:
# Скалярное произведение двух векторов, представленных списками
def dotVec(d1, d2):
    intersection = set.intersection(set(d1.keys()), set(d2.keys()))
    value = 0
    for key in intersection:
        value += d1[key] * d2[key]
    return value

### Составление векторов предложений

In [8]:
# Список слов
all_words = set()
for sent in all_sentenses:
    for word in sent:
        all_words.add(word)

In [41]:
# Вектора представлены словарями (нет слова = 0)
vectors_sent = []
for sent in all_sentenses:
    words = {}
    for word in sent:
        words[word] = words.setdefault(word, 0) + 1
    
    vectors_sent.append(words)   

In [44]:
# Нормализация векторов
vectors_sent_norm = list(vectors_sent)
for i, vec in enumerate(vectors_sent):
    vectors_sent_norm[i] = normolizeVec(vec)

In [11]:
# idf
df = dict(zip(all_words, np.zeros(len(all_words))))
for vec in vectors_sent:
    words = vec.keys()
    for word in words:
        df[word] += 1
        
idf = dict(zip(df.keys(), np.log(float(len(all_words)) / np.array(df.values()))))

### Вектор запроса

In [12]:
queries = [u'Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами.',
           u'С собакой викингов ходят на лося.',
           u'Герои мультфильма стали индикаторами невесомости космического корабля «Арго».'
          ]

vectors_query = []

for query in queries:
    words = {}
    norm_words = textUrl.sentenseToWords(query)
    for word in norm_words:
        words[word] = words.setdefault(word, 0) + 1
    
    vectors_query.append(words)   

### Векторное ранжирование

In [13]:
def vecRanging(query, sentenses):
    rang = []
    for sent in sentenses:
        rang.append(dotVec(query, sent) / (lenVector(query) * lenVector(sent)))
    return sorted(zip(range(len(rang)), rang), key=itemgetter(1), reverse=True)

In [14]:
def getTopVec(num_query, ranging='vecRanging', N=7):
    if N == 'all':
        N = 1000000
    if ranging == 'vecRanging':
        serp = vecRanging(vectors_query[num_query], vectors_sent_norm)[:N]
    else:
        serp = tf_idfRanging(vectors_query[num_query], vectors_sent)[:N]
    return serp

In [15]:
def printSerp(serp, num_query):
    print u"Запрос: " + queries[num_query]
    print u"Топ выдачи:"
    for result in serp:
        print "\t" + all_real_sentenses[result[0]] + "\t" + str(result[1])
        print "\n"

In [16]:
for i in [0,1,2]:
    printSerp(getTopVec(i, ranging='vecRanging'), i)
    print '____________________________________'
    print

Запрос: Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами.
Топ выдачи:
	В середине 1920-х годов на французском рынке сырной продукции появилось множество имитаций «Смеющейся коровы» и разнообразных вариаций на тему этикетки Рабье — «Улыбающаяся корова» (фр. «La vache qui sourit»), «Говорящая корова» (фр. «La vache qui parle»), «Читающая корова» (фр. «La vache qui lit»), «Учёная корова» (фр. «La vache savante»), «Брыкающаяся корова» (фр. «La vache qui rue») и др., в ассортименте были представлены также «Смеющаяся обезьяна» (фр. «Le Singe qui rit»), «Смеющаяся коза» (фр. «La chèvre qui rit»), «Смеющийся кот» (фр. «Le chat qui rit») и др.[15][14]	0.288874152291


	<…> C собакой ещё проходит, но заставить смеяться корову!	0.258198889747


	Весёлая бурёнка (фр. La vache qui rit, с фр. — «Смеющаяся корова») — французский плавленый сыр, производимый группой компаний Bel Group.	0.237170824513


	Рисунок смеющейся коровы был сделан по мотивам виден

### TF-IDF ранжирование

In [17]:
def tf_idfRanging(query, sentenses):
    rang = []
    for sent in sentenses:
        tf_idf = {}
        for word in sent.keys():
            tf_idf[word] = np.log(sent[word] + 1) * idf[word]
        sc = dotVec(tf_idf, query)
        
        rang.append(sc / (lenVector(query) * lenVector(tf_idf)))
            
    return sorted(zip(range(len(rang)), rang), key=itemgetter(1), reverse=True)

In [18]:
for i in [0,1,2]:
    printSerp(getTopVec(i, ranging='tf_idf'), i)
    print '____________________________________'
    print

Запрос: Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами.
Топ выдачи:
	В середине 1920-х годов на французском рынке сырной продукции появилось множество имитаций «Смеющейся коровы» и разнообразных вариаций на тему этикетки Рабье — «Улыбающаяся корова» (фр. «La vache qui sourit»), «Говорящая корова» (фр. «La vache qui parle»), «Читающая корова» (фр. «La vache qui lit»), «Учёная корова» (фр. «La vache savante»), «Брыкающаяся корова» (фр. «La vache qui rue») и др., в ассортименте были представлены также «Смеющаяся обезьяна» (фр. «Le Singe qui rit»), «Смеющаяся коза» (фр. «La chèvre qui rit»), «Смеющийся кот» (фр. «Le chat qui rit») и др.[15][14]	0.27861798662


	<…> C собакой ещё проходит, но заставить смеяться корову!	0.22558305037


	Весёлая бурёнка (фр. La vache qui rit, с фр. — «Смеющаяся корова») — французский плавленый сыр, производимый группой компаний Bel Group.	0.217366945766


	Наиболее известными работами, по оценке критиков, стал

### Ручная разметка

'Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами'
'С собакой викингов ходят на лося.'
'Герои мультфильма стали индикаторами невесомости космического корабля «Арго».'

In [20]:
step = 10
from_ = 0

In [21]:
for sent in all_real_sentenses[from_:from_ + step]:
    print sent
    print 

Состояниеотпатрулирована


30 декабря 1864(1864-12-30)

Ла-Рош-сюр-Йон (Вандея), Франция

10 октября 1939(1939-10-10) (74 года)

Фавроль (Эндр), Франция

Франция Франция

график, карикатурист, аниматор

Бенжаме́н Рабье́ (фр. Benjamin Rabier; 30 декабря 1864[К 1], Ла-Рош-сюр-Йон, департамент Вандея — 10 октября 1939, Фавроль, департамент Эндр, Франция) — французский художник-график, литератор, книжный иллюстратор, автор комиксов, один из пионеров анималистической мультипликации[1][2][3].

Родился 30 декабря 1864 года в Ла-Рош-сюр-Йон в рабочей семье, родом из Эндра.

Мать, Мари Массон (фр. Marie Masson) была дочерью трактирщика, отец — помощником столяра.



In [22]:
# marks = ['0000000000',
#          '0000000000',
#          '0000000000'
#         ]
# with open('sentenses_marks', 'a') as sent_file:
#     for i, sent in enumerate(all_real_sentenses[from_:from_+step]):
#         line = "\t".join([sent.strip(), marks[0][i], marks[1][i], marks[2][i]]) + '\n'
#         sent_file.write(line.encode('utf8'))
# from_ += step

In [77]:
DF = pd.read_csv('sentenses_marks', header=None, sep="\t")

In [78]:
DF

,0,1,2,3
0,Состояниеотпатрулирована,0,0,0
1,30 декабря 1864(1864-12-30),0,0,0
2,"Ла-Рош-сюр-Йон (Вандея), Франция",0,0,0
3,10 октября 1939(1939-10-10) (74 года),0,0,0
4,"Фавроль (Эндр), Франция",0,0,0
5,Франция Франция,0,0,0
6,"график, карикатурист, аниматор",1,0,0
7,Бенжаме́н Рабье́ (фр. Benjamin Rabier; 30 дека...,2,0,0
8,Родился 30 декабря 1864 года в Ла-Рош-сюр-Йон ...,0,0,0
9,"Мать, Мари Массон (фр. Marie Masson) была доче...",0,0,0


In [27]:
for sent in DF[DF[1] == 2][0]:
    print sent
    print 

Бенжаме́н Рабье́ (фр. Benjamin Rabier; 30 декабря 1864[К 1], Ла-Рош-сюр-Йон, департамент Вандея — 10 октября 1939, Фавроль, департамент Эндр, Франция) — французский художник-график, литератор, книжный иллюстратор, автор комиксов, один из пионеров анималистической мультипликации[1][2][3].

В 1917 году рисунок Рабье выигрывает в конкурсе на эмблему грузовиков для перевозки мяса, объявленном Военным министерством[К 3].

По заказу владельцев винокуренного предприятия под Парижем, задумавших выпуск «американского аперитива» — ликёра из овса под названием «Пикотин»[К 7], Рабье выполнил рекламный плакат, изображающий двух ослов, пьющих ликёр.

Рисунок смеющейся коровы был сделан по мотивам виденной им на грузовиках для перевозки мяса в годы Первой мировой войны эмблемы «La Wachkyrie», выполненной художником Бенжаменом Рабье (название обыгрывает слова «валькирия» и «vache qui rit»).



In [28]:
for sent in DF[DF[2] == 2][0]:
    print sent
    print 

охотничья собака

Норвежский чёрный элкхунд, или норвежская чёрная лосиная лайка[1] (норв.

Название происходит от elg (с норв. — «лось») и hund (с норв. — «собака»).

Используется для охоты на лося[3] и других копытных[2].



In [29]:
for sent in DF[DF[3] == 2][0]:
    print sent
    print 

20 апреля 2017 года игрушечные Юра и Нюра стали индикаторами невесомости на российском пилотируемом космическом корабле «Союз МС-04» «Арго» (автор игрушек-индикаторов — И. К. Найденова, кукольник — Н. А. Охливанкина).

20 апреля 2017 года игрушечные персонажи мультфильма стали индикаторами невесомости на российском пилотируемом космическом корабле Союз МС — 04 «Арго» (автор игрушек-индикаторов — И. К. Найденова, кукольник — Н. А. Охливанкина).

Другими индикаторами невесомости стали фигурки персонажей мультфильма телестудии Роскосмоса «Космические Юра и Нюра»[28], которые после полёта космонавт планирует передать в детский онкологический центр[29].



### NDCG 

In [30]:
def DCG(ranks):
    dcg = 0
    for pos, rank in enumerate(ranks):
        dcg += rank / np.log2(pos + 2) 
    return dcg

In [31]:
def NDCG(ranks):
    # упорядочивание по релевантрности
    best_sort = sorted(ranks, reverse=True)
    dcg = DCG(ranks)
    idcg = DCG(best_sort)
    ndcg = dcg / idcg
    return ndcg

#### Без idf

In [32]:
ndcg_vec = []
for i in range(3):
    serp = getTopVec(i, ranging='vecRanging', N='all')
    order = zip(*serp)[0]
    ranks = []
    for row_idx in order:
        ranks.append(DF.loc[row_idx, i + 1])
    print 'NDCG fot query: \t\n "%s" \n ' %queries[i]
    ndcg_vec.append(NDCG(ranks))
    print ndcg_vec[-1]
    print '–––––––––––––––––––––––––'

NDCG fot query: 	
 "Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами." 
 
0.76879541851
–––––––––––––––––––––––––
NDCG fot query: 	
 "С собакой викингов ходят на лося." 
 
0.86442859264
–––––––––––––––––––––––––
NDCG fot query: 	
 "Герои мультфильма стали индикаторами невесомости космического корабля «Арго»." 
 
0.931534841078
–––––––––––––––––––––––––


#### C idf

In [33]:
ndcg_idf = []
for i in range(3):
    serp = getTopVec(i, ranging='tf_idf', N='all')
    order = zip(*serp)[0]
    ranks = []
    for row_idx in order:
        ranks.append(DF.loc[row_idx, i + 1])
    print 'NDCG fot query: \t\n "%s" \n ' %queries[i]
    ndcg_idf.append(NDCG(ranks))
    print ndcg_idf[-1]
    print '–––––––––––––––––––––––––'

NDCG fot query: 	
 "Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами." 
 
0.785079527857
–––––––––––––––––––––––––
NDCG fot query: 	
 "С собакой викингов ходят на лося." 
 
0.876216962845
–––––––––––––––––––––––––
NDCG fot query: 	
 "Герои мультфильма стали индикаторами невесомости космического корабля «Арго»." 
 
0.941810576266
–––––––––––––––––––––––––


### Среднее по запросам

#### Без idf

In [34]:
print np.mean(np.array(ndcg_vec))

0.854919617409


#### C idf

In [35]:
print np.mean(np.array(ndcg_idf))

0.867702355656


### Языковая модель

In [61]:
def getProb(q, doc):
    res = np.zeros(len(q.items()))
    for i, q_word in enumerate(q.keys()):
        for d_word, d_num in doc.items():
            if q_word == d_word:
                res[i] += d_num
        res[i] /= len(doc)
    return np.array(res)

In [94]:
def getRanks(q, docs, lamb=0.5):
    union_docs = {}
    for doc in docs:
        union_docs = {k : union_docs.get(k, 0) + doc.get(k, 0) for k in set(union_docs) | set(doc)}
        
    union_prob = getProb(q, union_docs)
    res = []
    for doc in docs:
        vecProb = (1 - lamb) * union_prob + lamb * getProb(q, doc) + \
                  np.array([10e-32] * len(q)) # сглаживание ненайденных слов
        res.append(np.prod(vecProb))
    
    return res

In [95]:
for num_query in range(3):
    rank = getRanks(vectors_query[num_query], vectors_sent)
    serp = sorted(zip(range(len(rank)), rank), key=itemgetter(1), reverse=True)
    order = zip(*serp)[0]
    ranks = []
    for row_idx in order:
        ranks.append(DF.loc[row_idx, num_query + 1])
    print 'NDCG fot query: \t\n "%s" \n ' %queries[num_query]
    ndcg_idf.append(NDCG(ranks))
    print ndcg_idf[-1]
    print '–––––––––––––––––––––––––'

NDCG fot query: 	
 "Французский художник добился успеха со смеющейся коровой (на илл.) и провалился с пьющими ослами." 
 
0.825624507667
–––––––––––––––––––––––––
NDCG fot query: 	
 "С собакой викингов ходят на лося." 
 
0.846001189251
–––––––––––––––––––––––––
NDCG fot query: 	
 "Герои мультфильма стали индикаторами невесомости космического корабля «Арго»." 
 
0.940799193495
–––––––––––––––––––––––––


In [96]:
np.mean(np.array(ndcg_idf))

0.78473649123298983